# Cutberto Arizabalo Nava

Data: https://www.kaggle.com/competitions/nfl-big-data-bowl-2020/data

In [ ]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2//spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark



Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state info

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
drive.mount("/content/gdrive")  
!pwd  # show current path 

%cd "/content/gdrive/MyDrive/IA95022/spark_ml"
!ls  # show current directory

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content
/content/gdrive/MyDrive/IA95022/spark_ml
train.csv


In [ ]:
#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.2-bin-hadoop3.2'

In [ ]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.appName('app').getOrCreate()
spark_session

In [ ]:
df_spark = spark_session.read.csv('train.csv', header=True, inferSchema=True)
df_spark

DataFrame[GameId: int, PlayId: bigint, Team: string, X: double, Y: double, S: double, A: double, Dis: double, Orientation: double, Dir: double, NflId: int, DisplayName: string, JerseyNumber: int, Season: int, YardLine: int, Quarter: int, GameClock: string, PossessionTeam: string, Down: int, Distance: int, FieldPosition: string, HomeScoreBeforePlay: int, VisitorScoreBeforePlay: int, NflIdRusher: int, OffenseFormation: string, OffensePersonnel: string, DefendersInTheBox: int, DefensePersonnel: string, PlayDirection: string, TimeHandoff: timestamp, TimeSnap: timestamp, Yards: int, PlayerHeight: string, PlayerWeight: int, PlayerBirthDate: string, PlayerCollegeName: string, Position: string, HomeTeamAbbr: string, VisitorTeamAbbr: string, Week: int, Stadium: string, Location: string, StadiumType: string, Turf: string, GameWeather: string, Temperature: int, Humidity: int, WindSpeed: string, WindDirection: string]

In [ ]:
df_spark.printSchema()

root
 |-- GameId: integer (nullable = true)
 |-- PlayId: long (nullable = true)
 |-- Team: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- S: double (nullable = true)
 |-- A: double (nullable = true)
 |-- Dis: double (nullable = true)
 |-- Orientation: double (nullable = true)
 |-- Dir: double (nullable = true)
 |-- NflId: integer (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- JerseyNumber: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- YardLine: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- GameClock: string (nullable = true)
 |-- PossessionTeam: string (nullable = true)
 |-- Down: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- FieldPosition: string (nullable = true)
 |-- HomeScoreBeforePlay: integer (nullable = true)
 |-- VisitorScoreBeforePlay: integer (nullable = true)
 |-- NflIdRusher: integer (nullable = true)
 |-- OffenseFormation: string (n

In [ ]:
from pyspark.ml.feature import VectorAssembler

featassembler = VectorAssembler(inputCols=[
 'X',
 'Y',
 'Orientation',
 'Season',
 "Down",
 "HomeScoreBeforePlay",
 "Temperature",
 "Distance",
 "Humidity",
 'Quarter'], outputCol = "Independent Features" ).setHandleInvalid("skip")
featassembler

VectorAssembler_f6736ec88f83

In [ ]:
result = featassembler.transform(df_spark)
result.show()

+----------+--------------+----+-----+-----+----+----+----+-----------+------+-------+-----------------+------------+------+--------+-------+---------+--------------+----+--------+-------------+-------------------+----------------------+-----------+----------------+----------------+-----------------+----------------+-------------+-------------------+-------------------+-----+------------+------------+---------------+-----------------+--------+------------+---------------+----+----------------+--------------+-----------+----------+--------------+-----------+--------+---------+-------------+--------------------+
|    GameId|        PlayId|Team|    X|    Y|   S|   A| Dis|Orientation|   Dir|  NflId|      DisplayName|JerseyNumber|Season|YardLine|Quarter|GameClock|PossessionTeam|Down|Distance|FieldPosition|HomeScoreBeforePlay|VisitorScoreBeforePlay|NflIdRusher|OffenseFormation|OffensePersonnel|DefendersInTheBox|DefensePersonnel|PlayDirection|        TimeHandoff|           TimeSnap|Yards|Play

In [ ]:
final_data = result.select("Independent Features", "Yards")
final_data.show()

+--------------------+-----+
|Independent Features|Yards|
+--------------------+-----+
|[73.91,34.84,81.9...|    8|
|[74.67,32.64,27.6...|    8|
|[74.0,33.2,3.01,2...|    8|
|[71.46,27.7,359.7...|    8|
|[69.32,35.42,12.6...|    8|
|[75.06,24.0,308.3...|    8|
|[74.11,16.64,357....|    8|
|[73.37,18.73,328....|    8|
|[56.63,26.9,344.7...|    8|
|[73.35,38.83,75.4...|    8|
|[74.15,28.9,342.5...|    8|
|[75.82,17.56,178....|    8|
|[74.78,33.21,178....|    8|
|[75.43,32.41,207....|    8|
|[75.9,25.12,133.0...|    8|
|[79.76,29.49,192....|    8|
|[76.47,36.91,112....|    8|
|[74.7,19.19,152.1...|    8|
|[78.75,30.53,161....|    8|
|[74.6,31.88,218.4...|    8|
+--------------------+-----+
only showing top 20 rows



In [ ]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(featuresCol = 'Independent Features', labelCol='Yards')
model = model.fit(final_data)

In [ ]:
model.intercept

-311.133102109866

In [ ]:
model.coefficients

DenseVector([0.0006, -0.0013, -0.0, 0.1559, 0.1478, 0.0023, -0.0056, 0.1329, -0.0033, -0.019])

In [ ]:
model.summary.pValues

[0.05052493190044283,
 0.2418417597906739,
 0.8019881393749151,
 0.0,
 0.0,
 0.03884097114727503,
 0.0,
 0.0,
 8.43769498715119e-15,
 0.0508568983287363,
 0.0]

In [ ]:
prediction_result = model.evaluate(final_data)
prediction_result.predictions.show()

/content/spark-3.2.2-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+-----+------------------+
|Independent Features|Yards|        prediction|
+--------------------+-----+------------------+
|[73.91,34.84,81.9...|    8| 3.418018160852398|
|[74.67,32.64,27.6...|    8|3.4225437798920666|
|[74.0,33.2,3.01,2...|    8|3.4218642049434607|
|[71.46,27.7,359.7...|    8| 3.420466498677797|
|[69.32,35.42,12.6...|    8|3.4157379542841113|
|[75.06,24.0,308.3...|    8| 3.428738089255546|
|[74.11,16.64,357....|    8| 3.437031855493842|
|[73.37,18.73,328....|    8|3.4343376052499366|
|[56.63,26.9,344.7...|    8|3.4124835132682847|
|[73.35,38.83,75.4...|    8| 3.412441547614776|
|[74.15,28.9,342.5...|    8| 3.420899643338714|
|[75.82,17.56,178....|    8|3.4404617294052287|
|[74.78,33.21,178....|    8|3.4188071163862332|
|[75.43,32.41,207....|    8|3.4197225216532843|
|[75.9,25.12,133.0...|    8|3.4312915662016508|
|[79.76,29.49,192....|    8|3.4266734240453616|
|[76.47,36.91,112....|    8| 3.416252072621603|
|[74.7,19.19,152.1...|    8| 3.438107128